In [3]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import pandas as pd
import numpy as np
# import string
import math

# import scipy.stats as sts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
# import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
# from sklearn import discriminant_analysis as disan
# from sklearn import calibration as calib
# from sklearn import linear_model as lm
# from sklearn import svm
# from sklearn import gaussian_process as gaup
# from sklearn import mixture as mix
# from sklearn import tree
# from sklearn import ensemble as ens

import tensorflow as tf
from tensorflow import keras

# from keras import models as kermdls
# from keras import layers as kerlrs
# from keras import metrics as kmetrics

# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe

# import pickle

# import nilearn as nl
# from nilearn import plotting, image
# from nilearn import datasets
# import nibabel as nb
import h5py

import os

import time

In [67]:
tfr_train = 'train.tfrecord'

In [79]:
def decode(serialized_example):
    feats = tf.io.parse_single_example(serialized_example, features={'x1': tf.io.FixedLenFeature([], tf.string),
                                                                     'x2': tf.io.FixedLenFeature([], tf.string),
                                                                     'x3': tf.io.FixedLenFeature([], tf.string),
                                                                     'y': tf.io.FixedLenFeature([], tf.string)})

    x1 = tf.io.decode_raw(feats['x1'], tf.float64)
    x1.set_shape((32*37*32*424))
    x1 = tf.reshape(x1, [32,37,32,424])
    
    x2 = tf.io.decode_raw(feats['x2'], tf.float64)
    x2.set_shape((1378))
    
    x3 = tf.io.decode_raw(feats['x3'], tf.float64)
    x3.set_shape((26))
    
    y = tf.io.decode_raw(feats['y'], tf.float64)
    y.set_shape((5))

    X = (x1, x2, x3)
    return X, y

In [80]:

raw_dataset = tf.data.TFRecordDataset(tfr_train, compression_type="GZIP")
dataset = raw_dataset.map(decode)
dataset

# for x, y in dataset.take(3):
#     print(x.shape, y.shape, x.numpy().sum() == y.numpy()[0], y.numpy()[:-1].sum() == y.numpy()[-1])

<MapDataset shapes: (((32, 37, 32, 424), (1378,), (26,)), (5,)), types: ((tf.float64, tf.float64, tf.float64), tf.float64)>

In [93]:

for i in dataset.take(1):
    print(i[0][2].numpy())
    d = i[0][0].numpy()

[ 0.0088762   0.0072098   0.010822    0.0019134  -0.0030107   0.017054
  0.012393    0.008965   -0.00045006  0.038598   -0.00033381  0.0077431
  0.011806    0.019139    0.044719    0.0060512   0.01096     0.025009
  0.015425   -0.007702    0.028465    0.0079052  -0.0028193  -0.013976
  0.023928    0.018252  ]


In [88]:
def inputs_gen(idxs):
    for idx in idxs:
        # FNC inputs
        df_fnc = pd.read_csv('00_Data/fnc_csv/{0}.csv'.format(idx), index_col=0)
        X_fnc = np.array(df_fnc.values).reshape(-1)

        # Loading inputs
        df_loading = pd.read_csv('00_Data/loading_csv/{0}.csv'.format(idx), index_col=0)
        X_loading = np.array(df_loading.values).reshape(-1)

        #MRI inputs
        patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(idx), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        k = 2
        ki_padding = 3

        arr_regions = []
        for i in range(patient_SM.shape[0]):
            sample_map = patient_SM[i,:,:,:]
            # padding MRI map
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                         (map_shape[1]//k + 1)*k - map_shape[1],
                         (map_shape[2]//k + 1)*k - map_shape[2])

            npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        # padding i-th region by 3 pixels
                        ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                        arr_regions.append(ki_region_padded)
        X_mri = np.stack(arr_regions, axis=3)

    #     X = (X_mri, X_fnc, X_loading)
        return X_mri, X_fnc, X_loading

In [90]:
X_mri, X_fnc, X_loading = inputs_gen(['11216'])

In [92]:
X_loading

array([ 0.0088762 ,  0.0072098 ,  0.010822  ,  0.0019134 , -0.0030107 ,
        0.017054  ,  0.012393  ,  0.008965  , -0.00045006,  0.038598  ,
       -0.00033381,  0.0077431 ,  0.011806  ,  0.019139  ,  0.044719  ,
        0.0060512 ,  0.01096   ,  0.025009  ,  0.015425  , -0.007702  ,
        0.028465  ,  0.0079052 , -0.0028193 , -0.013976  ,  0.023928  ,
        0.018252  ])

In [94]:
print(d==X_mri)

[[[[ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   ...
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]]

  [[ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   ...
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]]

  [[ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   ...
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]]

  ...

  [[ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   ...
   [ True  Tru